In [3]:
#Analyzing Artist Data from Nobody Gets It Records

#Important Imports
import spotipy as sp
import sys
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import numpy as np
import random
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import time
import sys
import urllib
from sklearn.preprocessing import MultiLabelBinarizer
import requests
from urllib.parse import urlencode
import base64
import webbrowser
import openpyxl

In [4]:
#Connecting to the Spotify Web API, should ask user to sign in if they haven't done so already 
client_id = "ce672507967540598c451a12f8bcead2"
client_secret = "933b62a83e284d80a6c292e53ba29719"

auth_headers = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "https://localhost:1234/callback",
    "scope": "user-library-read"
}

webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))
#If using local host, take code from the search bar and input it into the code string on the next cell 


True

In [5]:
code = 'AQCIqote9KhjlR1Wv0-F00vUNQRhYiMhfmlR_7xx33GlsX1hxEu0aR3yMAE11zxgTTnSKXsZz_9L3nO43_wAVJFKwjXle6AeS5grG9XnlfxN5f8rQB4KDT3GTXOvN7-H3ke9S6fuxhAgYyQf1FUJ6qzzFa0SLxU3VcPuv6bmjGMZWsM1ts1THc-2V4YBpD4CvAMFnXGR'

In [6]:
 #Authorizing the code and returning a token 
encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "https://localhost:1234/callback"
}

r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)

In [7]:
token = r.json()["access_token"]

print(token)

BQBmTfXSbSdP2il6bzJ3g59IgnDcSM8N6Uu7sb3VNa2qR1rX8EebvGWDSV7H5OPiJWlr7hiUM8_fpwJubKUfyQbuD2K4eCm-GTNybp7mRYvCLNl18exEqkQZhmW146ZpBdzhikpMDxCwMFwXwbQv5KF6GNVCZf_RCw1S8VScVaiPvkciqCYBPV4YHfHiRqja_5Jqrz64BL7i1NK3dw4


In [8]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
s = sp.Spotify(client_credentials_manager=client_credentials_manager)


In [9]:
list_of_artists = ['Krooked Kings', 'Tame Impala', 'Omar Apollo', 'The Moss', 'Ashes to Amber', 'Arctic Monkeys', 'Peach Pit', 'Master Piece', 'Beach Fossils', 
'Declan McKenna', 'Dayglow', 'Cage the Elephant', '5 Seconds of Summer', 'Catfish in the Bottlemen', 'Easy Life', 'Wallows', 'Mac Demarco', 'The Neighbourhood', 
'The Strokes', 'M83', 'Wet Leg', 'Foster the People', 'Peter McPoland', 'The Walters', 'Good Morning', 'Phoenix', 'Kings of Leon', 'DICE', 'Lovejoy', 
'The Beaches', 'Retrofile', 'Dominic Fike', 'James Droll', 'Ratboys', 'Teenage Fanclub', 'Sad Night Dynamite', 'Pacific Avenue', 'Diners', 'binki', 
'Royel Otis', 'Johnny Goth', 'Djo', 'JAWNY', 'Vacations', 'Ripe', 'Beach Weather', 'Goldpark', 'Yot Club', 'Prelow', 'mehro', 'Cigarettes After Sex', 
'GROUPLOVE', 'The Backseat Lovers', 'Young the Giant', 'Bleachers', 'Bad Suns', 'Still Woozy', 'boy pablo', 'HUNNY', 'Surf Curse']

In [10]:
def get_artist_id(artist):
    result = s.search(q = 'artist: ' + artist, type = 'artist')
    return result['artists']['items'][0]['id']

In [12]:
result = s.search(q = 'artist: ' + 'The Strokes', type = 'artist')
result

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3A+The+Strokes&type=artist&offset=0&limit=10',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0epOFNiUfyON9EYx7Tpr6V'},
    'followers': {'href': None, 'total': 5071148},
    'genres': ['alternative rock',
     'garage rock',
     'modern rock',
     'permanent wave',
     'rock'],
    'href': 'https://api.spotify.com/v1/artists/0epOFNiUfyON9EYx7Tpr6V',
    'id': '0epOFNiUfyON9EYx7Tpr6V',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab6761610000e5ebcaea403b29f6a09260b6a18a',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/ab67616100005174caea403b29f6a09260b6a18a',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/ab6761610000f178caea403b29f6a09260b6a18a',
      'width': 160}],
    'name': 'The Strokes',
    'popularity': 74,
    'type': 'artist',
    'uri': 'spotify:artist:0epOFNiUfyON9EYx7Tpr6V'},
   {'ex

In [152]:
list_artistIds = []
for artist in list_of_artists: 
    id = get_artist_id(artist)
    list_artistIds.append(id)


In [153]:
list_artist_followers = [] 
list_artist_artist_popularities = []
for id in list_artistIds:
    data = s.artists([id])
    list_artist_followers.append(data['artists'][0]['followers']['total'])
    list_artist_artist_popularities.append(data['artists'][0]['popularity'])


In [154]:
ids_df = pd.DataFrame({'artist_name': list_of_artists, 'artist_id': list_artistIds, 'artist_followers': list_artist_followers, 'artist_popularity': list_artist_artist_popularities})
ids_df

,artist_name,artist_id,artist_followers,artist_popularity
0,Krooked Kings,6PbMwLmbus5mZl93tX1lXE,69747,48
1,Tame Impala,5INjqkS1o8h1imAzPqGZBb,6613826,80
2,Omar Apollo,5FxD8fkQZ6KcsSYupDVoSO,814332,70
3,The Moss,4wPeDqxPKkBN50kr8XhEma,38843,46
4,Ashes to Amber,1VjLe0p8LUFKPbyKaQZAgu,12605,53
5,Arctic Monkeys,7Ln80lUS6He07XvHI8qqHH,20354203,86
6,Peach Pit,6fC2AcsQtd9h4BWELbbire,725566,63
7,Master Piece,6RqCL0KK7WJhR4g9oDODuk,3531,20
8,Beach Fossils,1bwUhKRmEkOZ1wuTnV9XjC,511165,59
9,Declan McKenna,2D4FOOOtWycb3Aw9nY5n3c,915301,65


In [156]:
ids_df.to_excel('important_metrics.xlsx')